In [1]:
import qpms
import numpy as np
from numpy import newaxis as nx
import math
import cmath
from scipy.constants import c, e as eV, hbar
s3 = math.sqrt(3)
J_scat = 3

In [2]:
epsilon_b = 2.3104
hexside = 375e-9
cdn = c/ math.sqrt(epsilon_b)
maxlayer=4

omegalist=np.linspace(0,5e15,200)
#omegalist=np.linspace(0,3,3.2)*eV/hbar
#omegalist = np.array([1.5,])*eV/hbar

bz_0 = np.array((0,0,))
bz_K1 = np.array((1.,0))*4*np.pi/3/hexside/s3
bz_K2 = np.array((1./2.,s3/2))*4*np.pi/3/hexside/s3
bz_M = np.array((3./4, s3/4))*4*np.pi/3/hexside/s3

# reciprocal lattice basis
B1 = 2* bz_K1 - bz_K2
B2 = 2* bz_K2 - bz_K1

In [3]:
kdensity = 100
k0Mlist = bz_0 + (bz_M-bz_0) * np.linspace(0,1,kdensity)[:,nx]
kMK1list = bz_M + (bz_K1-bz_M) * np.linspace(0,1,kdensity)[:,nx]
kK10list = bz_K1 + (bz_0-bz_K1) * np.linspace(0,1,kdensity)[:,nx]
k0K2list = bz_0 + (bz_K2-bz_0) * np.linspace(0,1,kdensity)[:,nx]
kK2Mlist = bz_K2 + (bz_M-bz_K2) * np.linspace(0,1,kdensity)[:,nx]
klist = np.concatenate((k0Mlist,kMK1list,kK10list,k0K2list,kK2Mlist), axis=0)
kxmaplist = np.concatenate((np.array([0]),np.cumsum(np.linalg.norm(np.diff(klist, axis=0), axis=-1))))

In [4]:
centers=np.array([bz_0,B1,B2,-B1,-B2,B2-B1,-B2+B1,-B2-B1,B2+B1,-2*B1,-2*B2,-2*B2-B1,-2*B1-B2,-2*B1-2*B2])
centers2=qpms.generate_trianglepoints(maxlayer, v3d = False, include_origin= True)*4*np.pi/3/hexside
rot90 = np.array([[0,-1],[1,0]])
centers2=np.dot(centers2,rot90)

In [5]:
#%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.path import Path
import matplotlib.patches as patches
cmap = matplotlib.cm.prism
colormax = np.amax(np.linalg.norm(centers2,axis=0))
for oi in range(omegalist.shape[0]):
    omega = omegalist[oi]
    f, axes = plt.subplots(1,2, figsize=(10,4.52))
    ax = axes[0]
    #sc = ax.scatter(k2list[:,0], k2list[:,1], c = np.clip(np.abs(minsvTElistarr[oi,...,minN]),0,1), lw=0)
    for center in centers2:
        circle=plt.Circle((center[0],center[1]),omega/cdn, facecolor='none', edgecolor=cmap(np.linalg.norm(center)/colormax),lw=1)
        ax.add_artist(circle)
    verts = [(math.cos(math.pi*i/3)*4*np.pi/3/hexside/s3,math.sin(math.pi*i/3)*4*np.pi/3/hexside/s3) for i in range(6 +1)]
    codes = [Path.MOVETO,Path.LINETO,Path.LINETO,Path.LINETO,Path.LINETO,Path.LINETO,Path.CLOSEPOLY,]
    path = Path(verts, codes)
    patch = patches.PathPatch(path, facecolor='none', edgecolor='black',  lw=1)
    ax.add_patch(patch)
    ax.set_xlim([-8e6,8e6])
    ax.set_ylim([-8e6,8e6])
    ax.set_xticks([]) 
    ax.set_yticks([]) 
    #ax.scatter(centers2[:,0], centers2[:,1])
    print(omega/cdn)
    #f.colorbar(sc,ax=ax)
    
    ax = axes[1]
    for center in centers2:
        ax.plot(kxmaplist, np.linalg.norm(klist-center,axis=-1)*cdn, '-', color=cmap(np.linalg.norm(center)/colormax))

    #ax.set_xlim([np.min(kxmlarr),np.max(kxmlarr)])
    #ax.set_ylim([np.min(omegalist),np.max(omegalist)])
    xticklist = [0, kxmaplist[len(k0Mlist)-1], kxmaplist[len(k0Mlist)+len(kMK1list)-1], kxmaplist[len(k0Mlist)+len(kMK1list)+len(kK10list)-1], kxmaplist[len(k0Mlist)+len(kMK1list)+len(kK10list)+len(k0K2list)-1], kxmaplist[len(k0Mlist)+len(kMK1list)+len(kK10list)+len(k0K2list)+len(kK2Mlist)-1]]
    ax.set_xticks(xticklist)
    for xt in xticklist:
        ax.axvline(xt, ls='dotted', lw=0.3,c='k')
    ax.set_xticklabels(['Γ', 'M', 'K', 'Γ', 'K\'','M'])
    ax.axhline(omega, c='black')
    ax.set_ylim([0,5e15])
    ax2 = ax.twinx()
    ax2.set_ylim([ax.get_ylim()[0]/eV*hbar,ax.get_ylim()[1]/eV*hbar])
    f.savefig("diforder-hex-%.4f.pdf"%(omega/eV*hbar,))


0.0
127391.31273899274
254782.62547798548
382173.9382169783
509565.25095597096
636956.5636949637
764347.8764339566
891739.1891729492
1019130.5019119419
1146521.814650935
1273913.1273899274
1401304.4401289201
1528695.752867913
1656087.0656069056
1783478.3783458984
1910869.6910848913
2038261.0038238838
2165652.3165628766
2293043.62930187
2420434.942040862


/u/46/necadam1/unix/miniconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


2547826.254779855
2675217.567518848
2802608.8802578403
2930000.1929968335
3057391.505735826
3184782.8184748185
3312174.131213811
3439565.443952804
3566956.7566917967
3694348.06943079
3821739.3821697826
3949130.6949087754
4076522.0076477677
4203913.320386761
4331304.633125753
4458695.945864745
4586087.25860374
4713478.571342732
4840869.884081724
4968261.196820717
5095652.50955971
5223043.822298703
5350435.135037696
5477826.447776688
5605217.760515681
5732609.073254674
5860000.385993667
5987391.698732659
6114783.011471652
6242174.324210645
6369565.636949637
6496956.94968863
6624348.262427622
6751739.575166616
6879130.887905608
7006522.200644601
7133913.513383593
7261304.826122586
7388696.13886158
7516087.451600572
7643478.764339565
7770870.077078558
7898261.389817551
8025652.702556543
8153044.015295535
8280435.3280345285
8407826.640773522
8535217.953512514
8662609.266251506
8790000.578990499
8917391.89172949
9044783.204468485
9172174.51720748
9299565.829946471
9426957.142685464
9554348.4

In [6]:
import itertools
list(itertools.chain([1, 3, i] for i in range(4)))

[[1, 3, 0], [1, 3, 1], [1, 3, 2], [1, 3, 3]]